In [1]:
import numpy as np 
import pandas as pd
import cv2
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
directory = r"Agricultural-crops"
def get_folder_names(directory_path):
    folder_names = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]
    return folder_names

directory_path = directory
categories = get_folder_names(directory_path)

In [3]:
img_data = []

for catago in categories:
    folder = os.path.join(directory, catago)
    label = categories.index(catago)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        try:
            img_array = cv2.imread(img_path)
            img_array = cv2.resize(img_array, (100, 100))
            if img_array is not None and not img_array.size == 0:
                img_data.append([img_array, label])
            else:
                print(f"Error loading or resizing image: {img_path}")
        except Exception as e:
            print(f"Error processing image {img_path}: {str(e)}")

random.shuffle(img_data)

In [15]:
X = []
Y = []
for features, labels in img_data:
    X.append(features)
    Y.append(labels)
    
X = np.array(X)
Y = np.array(Y)
X = X / 255

from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y, num_classes=30)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


# Update the model
cnn = Sequential()
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=X.shape[1:], activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Flatten())
cnn.add(Dense(units=64, activation="relu"))
cnn.add(Dense(units=30, activation="softmax"))
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
cnn.fit(X, Y, epochs=10, validation_split=0.2)

Epoch 1/10
21/21 [==============================] - 10s 285ms/step - loss: 3.4792 - accuracy: 0.0362 - val_loss: 3.4104 - val_accuracy: 0.0241
Epoch 2/10
21/21 [==============================] - 6s 264ms/step - loss: 3.3598 - accuracy: 0.0679 - val_loss: 3.3972 - val_accuracy: 0.0241
Epoch 3/10
21/21 [==============================] - 5s 263ms/step - loss: 3.2105 - accuracy: 0.1116 - val_loss: 3.3367 - val_accuracy: 0.0542
Epoch 4/10
21/21 [==============================] - 5s 228ms/step - loss: 2.9265 - accuracy: 0.1870 - val_loss: 3.1945 - val_accuracy: 0.1265
Epoch 5/10
21/21 [==============================] - 5s 223ms/step - loss: 2.5489 - accuracy: 0.2836 - val_loss: 3.1997 - val_accuracy: 0.1205
Epoch 6/10
21/21 [==============================] - 5s 224ms/step - loss: 2.1038 - accuracy: 0.4268 - val_loss: 3.1603 - val_accuracy: 0.1506
Epoch 7/10
21/21 [==============================] - 5s 221ms/step - loss: 1.6412 - accuracy: 0.5732 - val_loss: 3.2908 - val_accuracy: 0.1386
Epoch

In [19]:
def predict_crop(image):
    test = cv2.imread(image)
    test = cv2.resize(test, (100, 100))

# Ensure image has 3 color channels
    test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)

# Normalize pixel values to be between 0 and 1
    test = test / 255.0

# Add batch dimension
    test = np.expand_dims(test, axis=0)

# Perform prediction
    value = cnn.predict(test)

    output = np.argmax(value)

# Print the predicted class name
    return categories[output]

In [20]:
predict_crop('makhana.jpg')

1/1 [==============================] - 0s 53ms/step


'Fox_nut(Makhana)'